# Separador e Extratos de texto de sentenças do Eproc SC

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2
import os
import re

## Configuração dos Diretórios de Entrada e Saída

* O diretório de entrada é o que possui os arquivos PDFs com várias sentenças, e apenas isso
* O diretório de saída será onde as sentenças separadas serão armazenadas, um PDF por sentença

In [ ]:
input_folder = "input"

input_files = os.listdir(input_folder)
print(f'Todos arquivos no diretório de entrada: {input_files}')

In [ ]:
# Remove qualquer arquivo que não seja PDF da lista
for f in input_files:
    if not f.endswith('.pdf'):
        input_files.remove(f)

print(f'PDFs do diretório de entrada: {input_files}')

In [ ]:
# Diretório de saída, onde as sentenças separadas serão armazenadas
output_folder = "output"

# Verificar se o diretório não existe
if not os.path.exists(output_folder):
    # Criar o diretório de saída se não existir
    os.makedirs(output_folder)
    print(f"Diretório de saída: `{output_folder}` criado com sucesso!")
else:
    # Se existir, só avisa que já existe
    print(f"O diretório de saída `{output_folder}` já existe!")


In [ ]:
# Função para imprimir o texto na cor vermelha
def red(text):
    return f"\033[91m{text}\033[0m"

## Parte 1 - Divisão das sentenças em arquivos individuais

### Processamento de um arquivo de entrada

In [ ]:
# Função que lê um arquivo PDF (pdf_reader já aberto) e divide em múltiplos PDFs
# de acordo com a lista de páginas (pages) marcadas.
# O nome dos arquivos gerados são os números dos processos (sentence_ids)

def split_sentence_files(pdf_reader,pages,sentence_ids):
    # Marcador da primeira página 
    start_page = 0
    for doc_index, end_page in enumerate(pages):
        sentence_id = sentence_ids[doc_index]
        output_filename = f"{output_folder}/{sentence_id}.pdf"
        counter = 1
        while os.path.exists(output_filename): # Se o arquivo já existe, ou seja, uma sentença com o mesmo número de processo, gera um nome seguido de um número
            print(red(f'O arquivo `{output_filename}` já existe'))
            output_filename = f"{output_folder}/{sentence_id}({counter}).pdf"
            print(f'Criando o arquivo: {output_filename}')
            counter += 1
        
        
        print('- Em processamento: [',doc_index+1,"/",len(pages),"] : [", start_page, ",", end_page , '] - ', sentence_ids[doc_index], ":",output_filename)
        # Escreve arquivos na saída
        pdf_writer = PyPDF2.PdfWriter()
        pdf_writer.append(fileobj=pdf_reader, pages=(start_page, end_page))
        output_file = open(output_filename, "wb")
        pdf_writer.write(output_file)
        pdf_writer.close()
        output_file.close()
        
        start_page = end_page

### Processamento em lote dos arquivos de entrada

Para cada arquivo de entrada, realiza a operação de divisão

In [ ]:
# Processamento em lote de todos os arquivos no diretório de entrada!
# Para cada arquivo no diretório de entrada, divida-os em múltiplos no diretório de saída!

sentence_set = set() # Conjunto para identificar se alguma sentença repete!
sentence_ids_files = dict()

for file in input_files:
    filename = f'{input_folder}/{file}'
    print(f'>>>>> Processando arquivo: `{filename}`. AGUARDE...')
    # Ler arquivo de entrada
    pdf_fileobj = open(filename,'rb')
    
    # Criar o leitor do arquivo de entrada
    pdf_reader = PyPDF2.PdfReader(pdf_fileobj)
    
    # Lista com as marcações das páginas de início e final de sentença
    pages_divisors = [] # Vetor com os números das páginas que iniciam cada sentença
    sentence_ids = []   # Vetor com o Identificador da sentença que é o número do processo
    for i in range(len(pdf_reader.pages)): # Para cada página do arquivo aberto
        page_obj=pdf_reader.pages[i]       # Lê a página
        page_text=page_obj.extract_text()  # Extrai o texto
        page_lines = page_text.split('\n') # Quebra as linhas
        page_first_line = page_lines[0]    # Obtém o texto da primeira linha

        # O que identifica a primeira página de cada sentença é o endereço da UFSC (baseado no arquivo de teste)
        if page_first_line == 'Av. Des. Vitor Lima, 183, fundos- Campus da UFSC - Bairro: Serrinha - CEP: 88040-400 - Fone: (48)3287-5019 - Email:':
            sentence_id = None 
            for line_index in range(-6,-1): # Busca identificador da sentença nas últimas 6 linhas do texto.
                sentence_id = re.match('[0-9]+-[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',page_lines[line_index])
                if sentence_id is not None: # Quando encontrar, para!
                    break

            if sentence_id.group() in sentence_set:
                for sentence_file in sentence_ids_files:
                    if sentence_id.group() in sentence_ids_files[sentence_file]:
                        print(red(f'A sentença {sentence_id.group()} já está no conjunto, pois apareceu no arquivo: {sentence_file}!'))
                
            sentence_set.add(sentence_id.group())
            pages_divisors.append(i)                  # Marca a página de início da sentença
            sentence_ids.append(sentence_id.group())  # O identificador da sentença (número do processo) que está no rodapé da página é registrado

    sentence_ids_files[f'{file}'] = sentence_ids
    pages_divisors.pop(0) # Remove o marcador de página 0, pois página que inícia a primeira sentença sempre será a primeira
    pages_divisors.append(len(pdf_reader.pages)) # Adiciona a última página do arquivo na lista de páginas.

    print(f'## Páginas que dividem as sentenças no arquivo `{file}`: {pages_divisors}')
    print(f'## Número dos processos do arquivo `{file}`: {sentence_ids}')
    split_sentence_files(pdf_reader,pages_divisors,sentence_ids) # Processa o arquivo
    pdf_fileobj.close()
    print()

print(red("\n\n######## CONCLUÍDO ########"))

In [ ]:
print('Sentenças encontradas em cada arquivo!')
sentence_ids_files

# Parte 2 - Extração de Texto

Lê os arquivos gerados no processamento em lote da etapa anterior, extrai o texto dos arquivos e gera um arquvo TXT apenas com o texto útil dos arquivos.

O texto útil é resultado da:
1. Remoção dos cabeçalhos
2. Ignorar rodapé

In [ ]:
# Usando o diretório de saída do processamento anterior,
# pois é nele que estão todos os PDFs das sentenças individualizadas
# para a extração de texto.
input_folder = "output"  
input_files = os.listdir(input_folder)

# Remove qualquer arquivo que não seja PDF da lista
for f in input_files:
    if not f.endswith('.pdf'):
        input_files.remove(f)

print(f'Há `{len(input_files)}` arquivos para extração de texto no diretório: {input_folder}.')

In [ ]:
# A spaCy não foi adotada na estratégia de geração manual, pode ser ignorada
!pip install spacy
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
from spacy.lang.pt import Portuguese

nlp = Portuguese()
# Criando o objeto spacy
#nlp = spacy.load("pt_core_news_sm")
# Create the pipeline 'sentencizer' component
sbd = nlp.add_pipe('sentencizer')

def tokenize_sentences(text):
    # nlp object is used to create documents with linguistic annotations
    doc = nlp(text)
    sentence_list =[]
    for sentence in doc.sents:
        sentence_list.append(sentence.text)
    
    return sentence_list

In [ ]:
roman_numbers = ('I.','II.','III.','IV.','V.','VI.','VII.','VIII.','IX.','X.', 
                 'XI.','XII.','XIII.','XIV.','XV.','XVI.','XVII.','XVIII.','XIX.','XX.',
                 'XXI.','XXII.','XXIII.','XXIV.','XXV.','XXVI.','XXVII.','XXVIII.','XXIX.','XXX.',

                 'I -','II -','III -','IV -','V -','VI -','VII -','VIII -','IX -','X -', 
                 'XI -','XII -','XIII -','XIV -','XV -','XVI -','XVII -','XVIII -','XIX -','XX -',
                 'XXI -','XXII -','XXIII -','XXIV -','XXV -','XXVI -','XXVII -','XXVIII -','XXIX -','XXX -'
                )

for file_index, file in enumerate(input_files):
    filename = f'{input_folder}/{file}'
    print(f'>>>>> Processando arquivo [{file_index+1}/{len(input_files)}]: `{filename}`. AGUARDE...')
    # Ler arquivo de entrada
    pdf_fileobj = open(filename,'rb')
    
    # Criar o leitor do arquivo de entrada
    pdf_reader = PyPDF2.PdfReader(pdf_fileobj)

    txt_content = ''
    txt_content2 = ''
    txt_content3 = ''
    
    piece_text = ''
    for p in range(len(pdf_reader.pages)): # Para cada página do arquivo aberto
        page_obj = pdf_reader.pages[p]      # Lê a página
        page_text = page_obj.extract_text() # Extrai o texto
        page_lines = page_text.split('\n')  # Quebra as linhas

        
        for line,text in enumerate(page_lines):
            if p == 0: # Primeira página do documento
                if line in [0,1]: # Primeiras duas linhas da primeira página: Cabeçalho com informações da UFSC
                    continue
                if line == 2:      # Terceira linha da primeira página: Informação do Tribunal e Número do Processo
                    txt_content += text.replace("PROCEDIMENT O", "PROCEDIMENTO") + '\n\n'
                    txt_content2 += text.replace("PROCEDIMENT O", "PROCEDIMENTO") + '\n\n'
                    txt_content3 += text.replace("PROCEDIMENT O", "PROCEDIMENTO") + '\n\n'
                    continue
                if text.strip().lower().startswith(('réu','autor','sentença')):
                    txt_content += text + '\n\n'
                    txt_content2 += text + '\n\n'
                    txt_content3 += text + '\n\n'
                    continue

            if re.match('^([0-9]+-[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)',text): # Encontrou o ID do processo no início da linha
                break          # Ao encontrar o ID de sentença (número do processo que fica no final de cada da página) - para de processar a página

            if text.startswith(roman_numbers) or re.match('^([0-9]+\.){1,3} ',text):
                if piece_text:
                    #print("\nImprimindo pedaço")
                    #print(piece_text)
                    #print(red('Sentenças'))
                    sentences = tokenize_sentences(piece_text)
                    txt_content2 += '\n'.join(sentences)
                    #print(sentences)

                txt_content += f'\n\n{text}\n\n'
                txt_content2 += f'\n\n{text}\n\n'
                txt_content3 += f'\n\n{text}\n\n'
                
                piece_text = ''
                continue

            if p == len(pdf_reader.pages)-1: # Se for a última página
                if text.startswith('Signatário'):
                    txt_content += f'\n{text}\n'  # Para pegar apenas o nome do Juiz: text.split(":")[1].strip()
                    sentences = tokenize_sentences(piece_text)
                    txt_content2 += '\n'.join(sentences)
                    txt_content2 += f'\n{text}\n'

                    txt_content3 += f'\n{text}\n'
                    continue
            piece_text += text + ' '
            txt_content += text + '\n'
            if re.match('^[A-Z]',text):
                if txt_content3.rstrip().endswith(('.', ':')):
                    txt_content3 += '\n' + text
                else:
                    txt_content3 += ' ' + text
            else:
                txt_content3 += ' ' + text
                    
            #print(f'[{line}] : {text}')
        # Processar linha-a-linha
        #print()

    pdf_fileobj.close()

#    f = open(f'{input_folder}/{file[:-4]}-all.txt', "w") # Gerar versão linha-a-linha separada, sem nenhum tratamento.
#    f.write(txt_content)
#    f.close()
#    f = open(f'{input_folder}/{file[:-4]}-v2.txt', "w")  # Gerar separação com tokenizador PT da spaCy
#    f.write(txt_content2)
#    f.close()
    f = open(f'{input_folder}/{file[:-4]}.txt', "w")     # Gerar tentativa manual de formatação.
    f.write(txt_content3)
    f.close()
#    print(f'TXT arquivo [{file}]: \n{txt_content}')

print(red('##### CONCLUÍDO #####'))